In [1]:
import os
import requests
import pandas as pd

In [2]:
rel = pd.read_csv('../0_data/manual/drugcentral_rel_06212018.csv')
syn = pd.read_csv('../0_data/manual/drugcentral_syn_06212018.csv')
ids = pd.read_csv('../0_data/manual/drugcentral_ids_06212018.csv')

In [3]:
rel.head()

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid
0,173432,965,40249340,indication,Malignant tumor of breast,C0006142,Malignant tumor of breast,T191,254837009.0
1,173433,318,21000716,indication,Gout,C0018099,Gout,T047,90560007.0
2,173434,318,21003276,indication,Hyperuricemia,C0740394,Hyperuricemia,T047,35885006.0
3,173435,1031,21001432,indication,Hypertensive disorder,C0020538,Hypertensive disorder,T047,38341003.0
4,173397,5226,21001068,indication,Tardive dyskinesia,C0686347,Tardive dyskinesia,T047,102449007.0


In [4]:
syn.head()

,syn_id,id,name,preferred_name,parent_id,lname
0,22490,5253.0,RPX-7009,NaN,NaN,rpx-7009
1,22493,5003.0,insulin (human),NaN,NaN,insulin (human)
2,22494,5191.0,insulin (pork),NaN,NaN,insulin (pork)
3,22495,5192.0,insulin (beef),NaN,NaN,insulin (beef)
4,22496,1762.0,methyldopa (levorotatory),NaN,NaN,methyldopa (levorotatory)


In [5]:
ids.head()

,id,identifier,id_type,struct_id,parent_match
0,1214712,D11040,KEGG_DRUG,5278,NaN
1,1214713,9745,IUPHAR_LIGAND_ID,5271,NaN
2,1214714,9088,IUPHAR_LIGAND_ID,3911,NaN
3,1214715,CHEBI:43362,CHEBI,5229,NaN
4,1214716,CHEBI:63452,CHEBI,5231,NaN


In [6]:
ids['id_type'].unique()

array(['KEGG_DRUG', 'IUPHAR_LIGAND_ID', 'CHEBI', 'DRUGBANK_ID', 'VUID',
       'NUI', 'UMLSCUI', 'ChEMBL_ID', 'UNII', 'INN_ID', 'SECONDARY_CAS_RN',
       'PUBCHEM_CID', 'RXNORM', 'NDDF', 'MMSL', 'VANDF', 'NDFRT',
       'SNOMEDCT_US', 'MESH_SUPPLEMENTAL_RECORD_UI', 'MESH_DESCRIPTOR_UI',
       'PDB_CHEM_ID'], dtype=object)

In [7]:
ids['id_type'].value_counts()

ChEMBL_ID                      6148
SNOMEDCT_US                    4976
UNII                           4391
PUBCHEM_CID                    4308
NDFRT                          4260
CHEBI                          3830
MMSL                           3782
INN_ID                         3589
KEGG_DRUG                      3577
NDDF                           3383
RXNORM                         2991
DRUGBANK_ID                    2929
UMLSCUI                        2835
MESH_SUPPLEMENTAL_RECORD_UI    2589
VANDF                          2181
NUI                            2044
VUID                           1790
SECONDARY_CAS_RN               1669
MESH_DESCRIPTOR_UI             1591
IUPHAR_LIGAND_ID               1395
PDB_CHEM_ID                     727
Name: id_type, dtype: int64

In [8]:
import functools
from wikidataintegrator.wdi_core import WDItemEngine
from tqdm import tqdm

endpoint='http://avalanche.scripps.edu:9999/bigdata/sparql'

def parse_result_uris(result):
    for c in result:
        if 'Label' not in c:
            idx = result[c].str.startswith('http://www.wikidata.org/entity')
            if sum(idx) != 0:
                result[c][idx] = result[c][idx].str.split('/', expand=True).iloc[:,-1]
    return result.drop_duplicates()

query_func = functools.partial(WDItemEngine.execute_sparql_query, endpoint=endpoint, as_dataframe=True)

def execute_sparql_query(query_text):
    # Enforce the proper column order
    col_order = query_text.split('\n')[1].split(' ?')[1:]
    return parse_result_uris(query_func(query_text))[col_order]

In [9]:
query = """
SELECT DISTINCT ?umlscui ?disease
WHERE {
  ?disease wdt:P2892 ?umlscui .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
}
"""

res = execute_sparql_query(query)
res.head(2)

,umlscui,disease
0,C0029516,Q170082
1,C0282687,Q162272


In [10]:
len(res)

23400

In [11]:
umls_to_wd = res.set_index('umlscui')['disease'].to_dict()

In [12]:
rel['disease_wd'] = rel['umls_cui'].map(umls_to_wd)

In [13]:
rel.query('relationship_name == "indication"')['umls_cui'].count(), rel.query('relationship_name == "indication"')['disease_wd'].count()

(8449, 5228)

In [14]:
struct_to_chembl = ids.query('id_type == "ChEMBL_ID"').set_index('struct_id')['identifier'].to_dict()

In [15]:
rel['comp_chembl'] = rel['struct_id'].map(struct_to_chembl)

In [16]:
rel.query('relationship_name == "indication"')['comp_chembl'].count()

10857

In [17]:
query = """
SELECT DISTINCT ?chembl ?compound
WHERE {
  ?compound wdt:P592 ?chembl .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGAGE],en" }
}
"""

res1 = execute_sparql_query(query)
res1.head(2)

,chembl,compound
0,CHEMBL1224,Q342745
1,CHEMBL1201589,Q415392


In [18]:
len(res1)

49621

In [19]:
chembl_to_wd = res1.set_index('chembl')['compound'].to_dict()

In [20]:
rel['comp_wd'] = rel['comp_chembl'].map(chembl_to_wd)

In [21]:
rel.query('relationship_name == "indication"')['comp_wd'].count()

8834

In [22]:
gs = rel.query('relationship_name == "indication"').dropna(subset=['comp_wd', 'disease_wd'])
len(gs)

4189

In [23]:
out_dir = '../2_pipeline/01_get_gold_standard/out'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
gs.to_csv(os.path.join(out_dir, 'gold_standard.csv'), index=False)